In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from time import time

/home/mariano/ENTER/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Set de entrenamiento y Test final

Cargamos los archivos necesarios

In [2]:
train = pd.read_csv('csv_files/modelo_final_featured.csv')
test_final = pd.read_csv('csv_files/test_final_featured.csv')

Transformamos las variables categóricas al tipo correspondiente, para reducir el uso de memoria

In [3]:
train['rango_edad'] = train['rango_edad'].astype('category')
train['sexo'] = train['sexo'].astype('category') 
train['nivel_estudios'] = train['nivel_estudios'].astype('category')
train['esta_estudiando'] = train['esta_estudiando'].astype('category')
train['tipo_de_trabajo'] = train['tipo_de_trabajo'].astype('category')
train['nivel_laboral'] = train['nivel_laboral'].astype('category')
train['nombre_zona'] = train['nombre_zona'].astype('category')
train['num_area'] = train['num_area'].astype('category')
train['post_por_nivel'] = train['post_por_nivel'].astype('category')
train['sepostulo'] = train['sepostulo'].astype('category')

In [4]:
test_final['rango_edad'] = test_final['rango_edad'].astype('category')
test_final['sexo'] = test_final['sexo'].astype('category') 
test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')
test_final['num_area'] = test_final['num_area'].astype('category')
test_final['post_por_nivel'] = test_final['post_por_nivel'].astype('category')

Nos guardamos los IDs necesarios para el submit

In [5]:
id_aviso_postulante = test_final['id']

Analizamos el set de entrenamiento y el test final

In [6]:
train.sample(5)

,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,descripcion,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,sepostulo,num_area,post_por_nivel
221898,1112466773,aDjY2W,4.0,1,4.0,0.0,<p>¿Buscas un trabajo de 6 horas? ¿Queres trab...,9,1,3,Call Center,0.0,4.0,0.0
930977,1112357563,PmVW14Y,3.0,1,4.0,0.0,<p>Seleccionamos Asesores Comerciales / Vende...,10,3,3,Salud,1.0,19.0,75.0
667706,1112359648,pzYWlpr,4.0,2,6.0,0.0,"<p>En Atos Argentina, empresa líder en servici...",10,3,3,Ventas,1.0,2.0,8.0
496382,1112218738,1BQJpN,4.0,2,2.0,0.0,<p>Nos encontramos en la búsqueda de un<strong...,10,3,3,Análisis Funcional,0.0,17.0,18.0
713299,1112452567,Bm5wjrz,2.0,1,4.0,1.0,<p>Importante Empresa Autopartista se encuentr...,10,3,3,Recursos Humanos,1.0,8.0,58.0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
idaviso            1000000 non-null int64
idpostulante       1000000 non-null object
rango_edad         1000000 non-null category
sexo               1000000 non-null category
nivel_estudios     1000000 non-null category
esta_estudiando    1000000 non-null category
descripcion        1000000 non-null object
tipo_de_trabajo    1000000 non-null category
nivel_laboral      1000000 non-null category
nombre_zona        1000000 non-null category
nombre_area        1000000 non-null object
sepostulo          1000000 non-null category
num_area           1000000 non-null category
post_por_nivel     1000000 non-null category
dtypes: category(10), int64(1), object(3)
memory usage: 41.0+ MB


In [8]:
test_final.sample(5)

,id,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,num_area,post_por_nivel
87499,87499,1112466838,Nz05OL4,1.0,2.0,4.0,1.0,Responsable de Canales,10.0,4.0,3.0,Comercial,2.0,2.0
8745,8745,1111791687,lDLoKj2,1.0,2.0,1.0,0.0,"ayudante de cocina/camarero eventuales, la m...",7.0,2.0,3.0,Gastronomia,9.0,7.0
14182,14182,1112094756,xka8kpK,2.0,2.0,2.0,0.0,Representante de Atencion al Cliente/ Banco,9.0,3.0,3.0,Call Center,4.0,24.0
95036,95036,1112487050,5ozmXz,2.0,1.0,4.0,1.0,Administrativo de Logística Eventual - CABA,10.0,3.0,3.0,Logística,11.0,4.0
20476,20476,1112256641,ZR0YJb,3.0,1.0,4.0,0.0,OPERARIO de BALANCINES Zona Lomas de Zamora GB...,10.0,3.0,3.0,Producción,3.0,0.0


In [9]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
rango_edad         100000 non-null category
sexo               100000 non-null category
nivel_estudios     100000 non-null category
esta_estudiando    100000 non-null category
titulo             100000 non-null object
tipo_de_trabajo    100000 non-null category
nivel_laboral      100000 non-null category
nombre_zona        100000 non-null category
nombre_area        100000 non-null object
num_area           100000 non-null category
post_por_nivel     100000 non-null category
dtypes: category(9), int64(2), object(3)
memory usage: 4.8+ MB


Nos guardamos los features que utilizaremos para los algoritmos de Machine Learning

In [10]:
# Features
f1 = 'rango_edad'
f2 = 'sexo'
f3 = 'nivel_estudios'
f4 = 'esta_estudiando'
f5 = 'tipo_de_trabajo'
f6 = 'nivel_laboral'
f7 = 'nombre_zona'
f8 = 'num_area'
f9 = 'post_por_nivel'

features = [f1,f2,f3,f4,f5,f6,f8,f9]

Particionamos el set de entrenamiento para realizar pruebas locales de hiper-parámetros antes de realizar cada submit

In [11]:
test_s = 0.25
random_s = 0

x = np.array(train[features])
y = np.array(train['sepostulo'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

Generamos el set de entrenamiento para el test final

In [12]:
x_test_final = np.array(test_final[features])

# Algoritmos probados

# 1. KNN

### Valores iniciales para K

In [ ]:
t0 = time()

print('VALORES INICIALES PARA K')

k_valores = [5,10,20,30,40,50]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES INICIALES PARA K
La precisión para k= 5 es: 0.815804
La precisión para k= 10 es: 0.82326


### Valores grid search para K

In [ ]:
t0 = time()

print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Predecimos el test final con el mejor k obtenido

In [ ]:
# Creamos el KNN
knn = KNeighborsClassifier(n_neighbors=mejor_k)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test_final)

### Submit File

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submits/submit_knn.csv', index=False)

submit['sepostulo'].value_counts()

# 2. Random Forest

### Set de entrenamiento

In [13]:
t0 = time()

# Parámetros óptimos
params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el Random Forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.789756
Tiempo de ejecución: 32.18067 seconds.


### Test final

#### Predicciones con valores binarios

In [14]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = random_forest.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 45.83240 seconds.


#### Predicciones con probabilidades

In [15]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = random_forest.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 39.08275 seconds.


### Submit files

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_random_forest.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_random_forest_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 3. Extra Trees

### Set de entrenamiento

In [16]:
t0 = time()

# Parámetros óptimos
params_optimos_et = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el extra_trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con nuestro set de entrenamiento
extra_trees.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = extra_trees.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.671444
Tiempo de ejecución: 32.77066 seconds.


### Test final

#### Predicciones con valores binarios

In [17]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = extra_trees.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 43.61266 seconds.


#### Predicciones con probabilidades

In [18]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = extra_trees.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 43.21765 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_extra_trees.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_extra_trees_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 4. Gaussian Naive Bayes

### Set de entrenamiento

In [19]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con nuestro set de entrenamiento
naive_bayes.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = naive_bayes.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.636404
Tiempo de ejecución: 0.83910 seconds.


### Test final

#### Predicciones con valores binarios

In [20]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = naive_bayes.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.45636 seconds.


#### Predicciones con probabilidades

In [21]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = naive_bayes.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 0.45705 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_naive_bayes.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_naive_bayes_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 5. Perceptron

### Set de entrenamiento

In [22]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con nuestro set de entrenamiento
perceptron.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = perceptron.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.787664
Tiempo de ejecución: 1.27687 seconds.


### Test final

#### Predicciones con valores binarios

In [23]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con la totalidad del set de datos
perceptron.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = perceptron.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 1.23521 seconds.


### Submit file

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_perceptron.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

# 6. Adaptive Boosting

### Set de entrenamiento

In [24]:
t0 = time()

# Parámetros óptimos
params_optimos_ab = {'n_estimators':50}

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.820452
Tiempo de ejecución: 31.23796 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = ada_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [42]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = ada_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 35.58527 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_ada_boost.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [43]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_ada_boost_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 7. Gradient Boosting

### Set de entrenamiento

In [27]:
t0 = time()

# Parámetros óptimos
params_optimos_gb = {'learning_rate':0.1, 'n_estimators':75, 'max_depth':4, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'subsample':0.5, 'max_features':'sqrt', 'warm_start':True}

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con nuestro set de entrenamiento
gra_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = gra_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.824944
Tiempo de ejecución: 35.10870 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = gra_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [44]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = gra_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 41.68665 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_gra_boost.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [45]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_gra_boost_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 8. Logistic Regression

### Set de entrenamiento

In [30]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con nuestro set de entrenamiento
log_reg.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = log_reg.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.759756
Tiempo de ejecución: 6.67571 seconds.


### Test final

#### Predicciones con valores binarios

In [31]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = log_reg.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 6.19293 seconds.


#### Predicciones con probabilidades

In [32]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = log_reg.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 6.12622 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_log_reg.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_log_reg_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 9. Decision Tree

### Set de entrenamiento

In [33]:
t0 = time()

# Parámetros óptimos
params_optimos_dt = {'criterion':'entropy', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}

# Creamos el logistic regression
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con nuestro set de entrenamiento
decision_tree.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = decision_tree.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.823076
Tiempo de ejecución: 247.43929 seconds.


### Test final

#### Predicciones con valores binarios

In [34]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = decision_tree.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 469.61542 seconds.


#### Predicciones con probabilidades

In [35]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = decision_tree.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 461.34604 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_decision_tree.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_decision_tree_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 10. Bagging

## 10.1 Con Logistic Regression

### Set de entrenamiento

In [36]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.758236
Tiempo de ejecución: 405.60441 seconds.


### Test final

#### Predicciones con valores binarios

In [37]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 388.88411 seconds.


#### Predicciones con probabilidades

In [38]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 371.44975 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_bagging_log_reg.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_bagging_log_reg_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

## 10.2 Con Decision Tree

### Set de entrenamiento

In [39]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_dt = {'base_estimator':DecisionTreeClassifier(**params_optimos_dt), 'n_estimators':10, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.83458
Tiempo de ejecución: 708.82964 seconds.


### Test final

#### Predicciones con valores binarios

In [41]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

KeyboardInterrupt: 

#### Predicciones con probabilidades

In [46]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 920.67132 seconds.


### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_bagging_decision_tree.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [47]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_bagging_decision_tree_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")

# 11. Voting

### Set de entrenamiento

In [ ]:
# Creamos los estimadores base que utilizaremos para el voting
estimador_1 = DecisionTreeClassifier(**params_optimos_dt)
estimador_2 = GradientBoostingClassifier(**params_optimos_gb)
estimador_3 = ExtraTreesClassifier(**params_optimos_et)

# Los agregamos a la lista de estimadores
estimadores = []
estimadores.append(('Naive Bayes', estimador_1))
estimadores.append(('Logistic Regression', estimador_2))
estimadores.append(('Random Forest', estimador_3))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con la totalidad del set de datos
voting.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = voting.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_voting.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

# 12. XGBoost

### Set de entrenamiento

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con nuestro set de entrenamiento
xgboost.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred_final = xgboost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred_final)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = xgboost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\                        
                            subsample=0.7,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = xgboost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
submit.to_csv('submits/submit_xgboost.csv', index=False)

In [ ]:
submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
submit_proba.to_csv('submits/submit_xgboost_proba.csv', index=False)

In [ ]:
no = submit_proba['sepostulo'] < 0.5
si = submit_proba['sepostulo'] >= 0.5

cant_no = submit_proba.loc[(no)].count()
cant_si = submit_proba.loc[(si)].count()

print("0   ", cant_no[1])
print("1   ", cant_si[1])
print("Name: sepostulo, dtype: int64")